In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="여행 목적지를 출력"),
    ResponseSchema(name="duration", description="여행 기간을 x박 x일로 표시"),
    ResponseSchema(name="budget", description="예산을 만원 단위로 출력"),
    ResponseSchema(name="rating", description="평점을 5점 만점으로 평가하여 출력"),
    ResponseSchema(name="activities", description="여행 목적지에서 활동한 것들을 리스트로 출력")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트 순서를 지켜 구조화된 형태로 출력해주세요.
리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

# 테스트 리뷰 데이터
review = input()

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행 목적지를 출력
	"duration": string  // 여행 기간을 x박 x일로 표시
	"budget": string  // 예산을 만원 단위로 출력
	"rating": string  // 평점을 5점 만점으로 평가하여 출력
	"activities": string  // 여행 목적지에서 활동한 것들을 리스트로 출력
}
```
===== 분석 결과 =====
{'activities': ['해운대 바다 구경', '자갈치시장 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4'}
